In [1]:
import os
from glob import glob
import json
from random import shuffle
from PIL import Image
from PIL import ImageDraw
import matplotlib.pyplot as plt
from PIL import ImageFont
import random
import numpy as np
from tqdm.autonotebook import tqdm 
import random 
from utils_datasets import get_font, get_samples

fonts = glob("fonts/**.ttf")
font_names = open("fonts/font_names.txt", 'r').read().splitlines()
font_colors = ["black"]
font_colors_ar = ["اسود"]

font_names_en = ['diwani decorated', 'diwani diacritized', 'diwani long', 'diwani standard',
                'kufi standard', 'kufi curved suqare', 'farisi standard', 'morrocan andulus',
                'rukaa bold', 'rukaa standard', 'rukaa fast', 'thuluth diwani', 'thuluth standard'
                ,'square standard', 'free bold', 'free standard', 'free long', 'mobili', 'managa'
                , 'aljazeera']

/var/tmp/ipykernel_12478/4149188987.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
!rm -r data_1m_mulfont_64x64_en_brown_no_scale

In [12]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             42G     0   42G   0% /dev
tmpfs           8.4G  688M  7.7G   9% /run
/dev/sda1       246G  126G  111G  54% /
tmpfs            42G  136K   42G   1% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            42G     0   42G   0% /sys/fs/cgroup
/dev/sda15      124M   11M  114M   9% /boot/efi
tmpfs           8.4G     0  8.4G   0% /run/user/1001
calliargen      1.0P     0  1.0P   0% /home/alyafey22/improved-diffusion/calliargen


In [13]:
max_words = 50000
W, H = (64, 64)

train_samples, valid_samples, test_samples = get_samples(max_words = max_words)
dataset = {'train': train_samples, 'valid': valid_samples, 'test': test_samples}

features = []
i = 0
path_to_data = f"data_1m_mulfont_{W}x{H}_en_brown_no_scale"
os.mkdir(path_to_data)
os.mkdir(f"{path_to_data}/valid")
os.mkdir(f"{path_to_data}/train")
os.mkdir(f"{path_to_data}/test")

In [14]:
from utils_datasets import get_font_fixed_scale
capts = {}
pbar = tqdm(total=max_words*20)
for split in dataset:
    for _, word in enumerate(dataset[split]):
        for _, font_type in enumerate(fonts):
            w, h = (W, H)
            #choose background and resize
            bg = np.random.choice(glob('fonts-bgs/06.jpg'))
            img = Image.open(bg)
            img = img.resize((W, H))
            #choose a font color
            font_color = np.random.choice(font_colors)
            font_color_ar = font_colors_ar[font_colors.index(font_color)]
            #get font description
            ft_idx = int(font_type[6:-4]) - 1
            font_name = font_names[ft_idx]
            #insert space
            if random.random() < 0.01 and len(word) > 2:
                rnd_idx = random.randint(1, len(word) - 1)
                word = word[:rnd_idx] + ' ' + word[rnd_idx:]

            draw = ImageDraw.Draw(img)
            font, w, h, x, y = get_font_fixed_scale(draw, word, font_type, width = W, height = H, margins =(10, 10))
            if font is None:
                continue
            draw.text((x, y), word, (0, 0, 0), font)
            file_name = f"{word}_{i:05d}"
            img.save(f"{path_to_data}/{split}/{file_name}.png")
            capts[file_name] = font_names_en[ft_idx]
            open(f"{path_to_data}/{split}/{file_name}.txt", 'w').write(word)
            i += 1
            pbar.update(1) 

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [15]:
capts[list(capts.keys())[0]]
with open(f"{path_to_data}/capts.json", "w") as fp:
    json.dump(capts,fp) 

In [22]:
"hello"

In [18]:
base_path = f"{path_to_data}/train"
for i, img_name in enumerate(os.listdir(base_path)):
    key, _ = img_name.split('.')
    print(image_name)
    if img_name.endswith(".png"):
        txt = open(f"{base_path}/{img_name[:-4]}.txt", "r").read()
        capt = capts.get(key)
        print(txt)
        print(capt)
        im = Image.open(f"{base_path}/{img_name}")
        plt.imshow(im)
        plt.show()
        if i > 50:
            break

In [23]:
base_path = f"../first_batch"
for i, img_name in enumerate(os.listdir(base_path)):
    key, _ = img_name.split('.')    
    if img_name.endswith(".png"):
        txt = open(f"{base_path}/{img_name[:-4]}.txt", "r").read()
        capt = open(f"{base_path}/{img_name[:-4]}_capt.txt", "r").read()
        print(txt)
        print(capt)
        im = Image.open(f"{base_path}/{img_name}")
        plt.imshow(im)
        plt.show()
        if i > 100:
            break

In [7]:
!pip install huggingface_hub

In [8]:
from huggingface_hub import login
login()

Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/alyafey22/.cache/huggingface/token
Login successful


In [10]:
from huggingface_hub import HfApi
api = HfApi()

In [11]:
api.upload_folder(
    folder_path=path_to_data,
    repo_id="arbml/CalliFontMedium",
    repo_type="dataset",
)

آلف_91921.png:   0%|          | 0.00/7.03k [00:00<?, ?B/s]

آلف_91922.png:   0%|          | 0.00/6.55k [00:00<?, ?B/s]

آلف_91920.png:   0%|          | 0.00/7.50k [00:00<?, ?B/s]

آلف_91924.png:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

Upload 99938 LFS files:   0%|          | 0/99938 [00:00<?, ?it/s]

...

ويكون_94954.png:   0%|          | 0.00/6.94k [00:00<?, ?B/s]

ويكون_94953.png:   0%|          | 0.00/6.91k [00:00<?, ?B/s]

ويكون_94955.png:   0%|          | 0.00/7.03k [00:00<?, ?B/s]

ويكون_94952.png:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

RuntimeError: Error while uploading 'test/ويكون_94944.png' to the Hub.